<a href="https://colab.research.google.com/github/Maxxx-VS/The-Founder/blob/master/49_5_Speech_Technologies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Установка библиотек
%%capture
!pip install SpeechRecognition pydub
!apt install ffmpeg

In [2]:
# Импорт библиотек
import os
import speech_recognition as sr
from pydub import AudioSegment
from pydub.generators import WhiteNoise
import numpy as np
from sklearn.metrics import accuracy_score

In [3]:
# Функция конвертации mp3 в wav
def convert_mp3_to_wav(input_mp3_path, output_wav_path=None, sample_rate=16000):

    # Загружаем MP3 файл
    audio = AudioSegment.from_mp3(input_mp3_path)

    # Устанавливаем частоту дискретизации
    audio = audio.set_frame_rate(sample_rate)

    # Экспортируем в WAV
    audio.export(output_wav_path, format="wav")
    print(f"Файл успешно конвертирован: {output_wav_path}")
    return output_wav_path

In [4]:
# Функция добавления шума
def add_noise_to_audio(input_path, output_path=None, noise_level=0.9):

    # Загружаем аудиофайл
    audio = AudioSegment.from_file(input_path)

    # Генерируем белый шум той же длительности, что и аудио
    noise = WhiteNoise().to_audio_segment(duration=len(audio))

    # Накладываем шум на оригинальное аудио
    noisy_audio = audio.overlay(noise, gain_during_overlay=noise_level*20)  # Умножаем на 20 для перевода в дБ

    # Если выходной путь не указан, добавляем '_noisy' к имени файла
    if output_path is None:
        base, ext = os.path.splitext(input_path)
        output_path = f"{base}_noisy{ext}"

    # Сохраняем результат
    noisy_audio.export(output_path, format=output_path.split('.')[-1])

    print(f"Файл с шумом успешно создан: {output_path}")
    return output_path

In [5]:
# Функция транскрибации аудио
def transcribe_audio(filename):

    r = sr.Recognizer()
    with sr.AudioFile(filename) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ru-RU")
        return text

In [14]:
# Конвертирум mp3 в wav
folder_path = '/content/'

# Создаем папку для чистых аудиофайлов
os.makedirs("clean", exist_ok=True)

# Получаем список всех файлов и папок
all_items = os.listdir(folder_path)

# Фильтруем только файлы (исключаем подпапки)
files = [item for item in all_items if os.path.isfile(os.path.join(folder_path, item))]
print(files)

for file in files:
    convert_mp3_to_wav(file, f"clean/{file[:-4]}_clean.wav")

['jenskie-golosovye.mp3', 'ne-mogu.mp3', 'utro.mp3', 'novoe.mp3', 'dobrogo-vremni-sutok.mp3']
Файл успешно конвертирован: clean/jenskie-golosovye_clean.wav
Файл успешно конвертирован: clean/ne-mogu_clean.wav
Файл успешно конвертирован: clean/utro_clean.wav
Файл успешно конвертирован: clean/novoe_clean.wav
Файл успешно конвертирован: clean/dobrogo-vremni-sutok_clean.wav


In [15]:
# Добавляем шум к записям
folder_path = '/content/clean/'

# Создаем папку для шумных аудиофайлов
os.makedirs("noise", exist_ok=True)

# Получаем список всех файлов и папок
all_items = os.listdir(folder_path)

# Фильтруем только файлы (исключаем подпапки)
files = [item for item in all_items if os.path.isfile(os.path.join(folder_path, item))]
print(files)

for file in files:
    add_noise_to_audio(f"{folder_path}{file}", f"noise/{file[:-10]}_noise.wav")

['utro_clean.wav', 'dobrogo-vremni-sutok_clean.wav', 'novoe_clean.wav', 'jenskie-golosovye_clean.wav', 'ne-mogu_clean.wav']
Файл с шумом успешно создан: noise/utro_noise.wav
Файл с шумом успешно создан: noise/dobrogo-vremni-sutok_noise.wav
Файл с шумом успешно создан: noise/novoe_noise.wav
Файл с шумом успешно создан: noise/jenskie-golosovye_noise.wav
Файл с шумом успешно создан: noise/ne-mogu_noise.wav


In [16]:
phrases = ["Доброго времени суток",
           "В чем ты сейчас одет",
           "Блин к сожалению не могу",
           "Для меня это что-то новое",
           "Доброе утро"]

results = []

In [17]:
for i in range(5):
    # Транскрибируем чистое аудио
    clean_text = transcribe_audio("/content/clean/dobrogo-vremni-sutok_clean.wav")

    # Транскрибируем шумное аудио
    noisy_text = transcribe_audio("/content/noise/dobrogo-vremni-sutok_noise.wav")

    # Оригинальный текст
    original_text = phrases[0].lower()


    results.append({"sample": i,
                    "original": original_text,
                    "clean_transcription": clean_text.lower() if clean_text else "",
                    "noisy_transcription": noisy_text.lower() if noisy_text else "",})

In [18]:
print(results)

[{'sample': 0, 'original': 'доброго времени суток', 'clean_transcription': 'доброго времени су', 'noisy_transcription': 'доброго времени'}, {'sample': 1, 'original': 'доброго времени суток', 'clean_transcription': 'доброго времени су', 'noisy_transcription': 'доброго времени'}, {'sample': 2, 'original': 'доброго времени суток', 'clean_transcription': 'доброго времени су', 'noisy_transcription': 'доброго времени'}, {'sample': 3, 'original': 'доброго времени суток', 'clean_transcription': 'доброго времени су', 'noisy_transcription': 'доброго времени'}, {'sample': 4, 'original': 'доброго времени суток', 'clean_transcription': 'доброго времени су', 'noisy_transcription': 'доброго времени'}]
